In [1]:
import PIL.Image as Image
import numpy as np
from tqdm import tqdm
from glob import glob
import random

import tensorflow as tf
from keras.models import Model
from keras.layers import add, Input
from keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D
from keras.layers import Concatenate, Activation
from keras.callbacks import EarlyStopping
from keras import backend

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DATA_DIR = '/kaggle/input/vesuvius-challenge-ink-detection'
HEIGHT = 4096 # Papyrus Height
PATCH_SIZE = 256
PATCH_HALFSIZE = PATCH_SIZE // 2
Z_START = 25 # Offset in z direction
Z_DIM = 16 # number of slices in z direction. Max value : 65 - Z_START
BATCH_SIZE = 32


In [3]:
def resize(img):
        curr_width, curr_height = img.size
        aspect_ratio = curr_width / curr_height

        width = int(HEIGHT * aspect_ratio)
        size = (width, HEIGHT)

        return img.resize(size)
    
def mask(split, index):
        img = Image.open(f'{DATA_DIR}/{split}/{index}/mask.png').convert('1')
        img = resize(img)

        return np.array(img, dtype= 'bool')

def label(split, index):
        img = Image.open(f'{DATA_DIR}/{split}/{index}/inklabels.png').convert('1')
        img = resize(img)

        return np.array(img, dtype= 'bool')

def volumes(split, index):
        z_slices_fname = sorted(glob(f'{DATA_DIR}/{split}/{index}/surface_volume/*.tif'))[Z_START : 
                                                                                          Z_START + Z_DIM]
        z_slices = []
        for filename in tqdm(z_slices_fname):
            img = Image.open(filename)
            img = resize(img)

            z_slice = np.array(img, dtype= 'float32')
            z_slices.append(z_slice)
        
        return np.stack(z_slices, axis= -1)
    
def load_sample(split, index):
        if split == 'train':
            return mask(split, index), label(split, index), volumes(split, index)
        return mask(split, index), volumes(split, index), None

In [4]:
mask_1, label_1, vol_1 = load_sample(split= 'train', index= 1)
mask_2, label_2, vol_2 = load_sample(split= 'train', index= 2)
mask_3, label_3, vol_3 = load_sample(split= 'train', index= 3)

100%|██████████| 16/16 [00:41<00:00,  2.62s/it]
/opt/conda/lib/python3.10/site-packages/PIL/Image.py:3176: DecompressionBombWarning: Image size (140973980 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████| 16/16 [00:33<00:00,  2.08s/it]


In [5]:
dev_folds = {
    'dev_1': {
        'train_volumes': [vol_1, vol_2],
        'train_labels': [label_1, label_2],
        'train_masks': [mask_1, mask_2],
        'validation_volume': vol_3,
        'validation_labels': label_3,
        'validation_mask': mask_3,
    },
    'dev_2': {
        'train_volumes': [vol_2, vol_3],
        'train_labels': [label_2, label_3],
        'train_masks': [mask_2, mask_3],
        'validation_volume': vol_1,
        'validation_labels': label_1,
        'validation_mask': mask_1,
    },
    'dev_3': {
        'train_volumes': [vol_1, vol_3],
        'train_labels': [label_1, label_3],
        'train_masks': [mask_1, mask_3],
        'validation_volume': vol_2,
        'validation_labels': label_2,
        'validation_mask': mask_2,
    }
}

In [6]:
def sample_random_location(shape):
    x = random.randint(PATCH_HALFSIZE, shape[0] - PATCH_HALFSIZE - 1)
    y = random.randint(PATCH_HALFSIZE, shape[1] - PATCH_HALFSIZE - 1)
    return (x, y)


def list_all_locations(mask, stride= PATCH_HALFSIZE):
    locations = []
    for x in range(PATCH_HALFSIZE, mask.shape[0] - PATCH_HALFSIZE, stride):
        for y in range(PATCH_HALFSIZE, mask.shape[1] - PATCH_HALFSIZE, stride):
            if mask[x, y]:
                locations.append((x, y))
    return locations


def extract_subvolume(location, volume):
    x = location[0]
    y = location[1]
    subvolume = volume[x - PATCH_HALFSIZE :x + PATCH_HALFSIZE,
                       y - PATCH_HALFSIZE :y + PATCH_HALFSIZE, :]
    subvolume = subvolume.astype('float32') / 65535.
    return subvolume


def extract_labels(location, labels):
    x = location[0]
    y = location[1]
    label = labels[x - PATCH_HALFSIZE :x + PATCH_HALFSIZE,
                    y - PATCH_HALFSIZE :y + PATCH_HALFSIZE]
    label = label.astype('float32')
    label = np.expand_dims(label, axis=-1)
    return label

# Generator For Training Data
def make_random_data_generator(volume, mask, labels):
    def data_generator():
        while True:
            loc = sample_random_location(mask.shape)
            if mask[loc[0], loc[1]]:
                subvolume = extract_subvolume(loc, volume)
                label = extract_labels(loc, labels)
                yield (subvolume, label)
    return data_generator

# Generator For Validation Data
def make_iterated_data_generator(volume, mask, labels= None):
    locations = list_all_locations(mask)
    def data_generator():
        for loc in locations:
            subvolume = extract_subvolume(loc, volume)
            if labels is None:
                yield subvolume
            else:
                label = extract_labels(loc, labels)
                yield (subvolume, label)
    return data_generator


def make_tf_dataset(gen_fn, labeled= True):
    if labeled:
        output_signature = (
            tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, Z_DIM), dtype=tf.float32),
            tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, 1), dtype=tf.float32),
        )
    else:
        output_signature = tf.TensorSpec(shape=(PATCH_SIZE, PATCH_SIZE, Z_DIM), dtype=tf.float32)
    ds = tf.data.Dataset.from_generator(
        gen_fn,
        output_signature=output_signature,
    )
    return ds.batch(BATCH_SIZE)

In [7]:
def make_datasets_for_fold(fold, train_augment_fn= None):
    train_volumes = fold['train_volumes']
    train_masks = fold['train_masks']
    train_labels = fold['train_labels']

    include_validation = 'validation_volume' in fold
    if include_validation:
        validation_volume = fold['validation_volume']
        validation_mask = fold['validation_mask']
        validation_labels = fold['validation_labels']

    all_train_ds = []
    for volume, mask, labels in zip(train_volumes, train_masks, train_labels):
        train_ds = make_tf_dataset(
            make_random_data_generator(volume, mask, labels),
            labeled= True,
        )
        all_train_ds.append(train_ds)
    train_ds = tf.data.Dataset.sample_from_datasets(all_train_ds)

    if train_augment_fn:
        train_ds = train_ds.map(train_augment_fn, num_parallel_calls= tf.data.AUTOTUNE)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

    if not include_validation:
        return train_ds

    val_ds = make_tf_dataset(
        make_iterated_data_generator(validation_volume, validation_mask, validation_labels),
        labeled= True,
    )
    return (train_ds, val_ds)

In [8]:
def trivial_baseline(dataset):
    total = 0
    matches = 0.
    for _, batch_label in dataset:
        matches += tf.reduce_sum(tf.cast(batch_label, 'float32'))
        total += tf.reduce_prod(tf.shape(batch_label))
    return 1. - matches / tf.cast(total, 'float32')

In [9]:
train_ds, val_ds = make_datasets_for_fold(fold= dev_folds['dev_1'])
score = trivial_baseline(val_ds)
print(f"Best validation score achievable trivially [fold 1]: {score * 100:.2f}% accuracy")

Best validation score achievable trivially [fold 1]: 87.46% accuracy


In [10]:
class Unet:

    def __init__(self, filter : int = 64) -> None:
        self.FILTER_NUM = filter
    
    def __conv__(self, input, num_filters):
        x = Conv2D(num_filters, 3, padding= 'same', kernel_regularizer= 'l2')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(num_filters, 3, padding= 'same', kernel_regularizer= 'l2')(input)
        x = BatchNormalization()(x)
        
        shortcut = Conv2D(num_filters, 1, padding= 'same')(input)
        shortcut = BatchNormalization()(shortcut)

        res_path = add([shortcut, x])
        res_path = Activation('relu')(res_path)

        return res_path
    
    def __encoder__(self, input, num_filters):
        s = self.__conv__(input, num_filters)
        p = MaxPooling2D(2)(s)

        return s, p

    def __decoder__(self, input, skip_features, num_filters):
        d = Conv2DTranspose(num_filters, 3, strides= 2, padding= 'same')(input)
        d = Concatenate()([d, skip_features])
        d = self.__conv__(d, num_filters)

        return d
    
    def gen_Unet(self, input_shape : any) -> Model:
        '''
        Generates A Residual Attention UNet Model \n
        '''
        input = Input(input_shape)

        # Encoder Block
        s1, p1 = self.__encoder__(input, self.FILTER_NUM)
        s2, p2 = self.__encoder__(p1, 2 * self.FILTER_NUM)
        s3, p3 = self.__encoder__(p2, 4 * self.FILTER_NUM)
        s4, p4 = self.__encoder__(p3, 8 * self.FILTER_NUM)

        # Base Block
        b = self.__conv__(p4, 16 * self.FILTER_NUM)

        # Decoder Block
        d1 = self.__decoder__(b, s4, 8 * self.FILTER_NUM)
        d2 = self.__decoder__(d1, s3, 4 * self.FILTER_NUM)
        d3 = self.__decoder__(d2, s2, 2 * self.FILTER_NUM)
        d4 = self.__decoder__(d3, s1, self.FILTER_NUM)

        # Output
        output = Conv2D(1, 1, activation= 'sigmoid', padding= 'same')(d4)
        
        model = Model(input, output)
        model.summary()
        return model

In [11]:
earlystop = EarlyStopping(monitor= 'val_loss', mode= 'min', patience= 5, start_from_epoch= 5)

In [12]:
backend.clear_session()

model = Unet()
model = model.gen_Unet(input_shape= (PATCH_SIZE, PATCH_SIZE, Z_DIM))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 256, 64  1088        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  9280        ['input_1[0][0]']                
                                )                                                             

In [13]:
model.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])

In [14]:
model.fit(train_ds, validation_data= val_ds, epochs= 25, steps_per_epoch= 100, callbacks= [earlystop])

Epoch 1/25
100/100 [==============================] - 111s 868ms/step - loss: 2.9009 - accuracy: 0.8130 - val_loss: 0.6393 - val_accuracy: 0.8746
Epoch 2/25
100/100 [==============================] - 84s 843ms/step - loss: 0.4995 - accuracy: 0.8286 - val_loss: 0.4883 - val_accuracy: 0.8746
Epoch 3/25
100/100 [==============================] - 84s 842ms/step - loss: 0.4597 - accuracy: 0.8397 - val_loss: 0.6028 - val_accuracy: 0.8746
Epoch 4/25
100/100 [==============================] - 84s 839ms/step - loss: 0.4575 - accuracy: 0.8442 - val_loss: 0.6594 - val_accuracy: 0.8746
Epoch 5/25
100/100 [==============================] - 84s 841ms/step - loss: 0.4434 - accuracy: 0.8483 - val_loss: 0.4635 - val_accuracy: 0.8700
Epoch 6/25
100/100 [==============================] - 84s 842ms/step - loss: 0.4416 - accuracy: 0.8510 - val_loss: 0.5297 - val_accuracy: 0.8746
Epoch 7/25
100/100 [==============================] - 84s 845ms/step - loss: 0.4501 - accuracy: 0.8509 - val_loss: 0.4846 - val_a

In [15]:
model.save('model.h5')

In [16]:
del vol_1
del vol_2
del vol_3

del mask_1
del mask_2
del mask_3

del label_1
del label_2
del label_3

del train_ds
del val_ds

backend.clear_session()
import gc
gc.collect()

1630

In [17]:
def compute_predictions_map(split, index):
    test_mask, test_volume, _ = load_sample(split=split, index=index)

    test_ds = make_tf_dataset(
        make_iterated_data_generator(test_volume, test_mask),
        labeled=False,
    )
    locations_ds = tf.data.Dataset.from_tensor_slices(
        list_all_locations(test_mask, stride=PATCH_SIZE)
    ).batch(BATCH_SIZE)

    predictions_map = np.zeros(test_volume.shape[:2] + (1,), dtype='float32')
    predictions_map_counts = np.zeros(test_volume.shape[:2] + (1,), dtype='int32')

    print(f'Compute predictions')

    for loc_batch, patch_batch in tqdm(zip(locations_ds, test_ds)):
        predictions = model.predict_on_batch(patch_batch)

        for (x, y), pred in zip(loc_batch, predictions):
            predictions_map[x - PATCH_HALFSIZE : x + PATCH_HALFSIZE, y - PATCH_HALFSIZE : y + PATCH_HALFSIZE, :] += pred
            predictions_map_counts[x - PATCH_HALFSIZE : x + PATCH_HALFSIZE, y - PATCH_HALFSIZE : y + PATCH_HALFSIZE, :] += 1
            
    predictions_map /= (predictions_map_counts + 1e-7)
    return predictions_map

In [18]:
predictions_map_a = compute_predictions_map(split='test', index='a')
predictions_map_b = compute_predictions_map(split='test', index='b')

100%|██████████| 16/16 [00:16<00:00,  1.05s/it]


Compute predictions


11it [00:07,  1.44it/s]
100%|██████████| 16/16 [00:26<00:00,  1.67s/it]


Compute predictions


5it [00:03,  1.47it/s]


In [19]:
'''
Resize Image To Original
'''
from skimage.transform import resize as resize_ski
import PIL.Image as Image

original_size_a = Image.open(DATA_DIR + "/test/a/mask.png").size
predictions_map_a = resize_ski(predictions_map_a, (original_size_a[1], original_size_a[0])).squeeze()

original_size_b = Image.open(DATA_DIR + "/test/b/mask.png").size
predictions_map_b = resize_ski(predictions_map_b, (original_size_b[1], original_size_b[0])).squeeze()

In [20]:
def rle(predictions_map, threshold):
    flat_img = predictions_map.flatten()
    flat_img = np.where(flat_img > threshold, 1, 0).astype(np.uint8)

    starts = np.array((flat_img[:-1] == 0) & (flat_img[1:] == 1))
    ends = np.array((flat_img[:-1] == 1) & (flat_img[1:] == 0))
    starts_ix = np.where(starts)[0] + 2
    ends_ix = np.where(ends)[0] + 2
    lengths = ends_ix - starts_ix
    return " ".join(map(str, sum(zip(starts_ix, lengths), ())))

In [21]:
THRESHOLD = 0.1

rle_a = rle(predictions_map_a, threshold=THRESHOLD)
rle_b = rle(predictions_map_b, threshold=THRESHOLD)
print("Id,Predicted\na," + rle_a + "\nb," + rle_b, file=open("submission.csv", "w"))